In [2]:
!pip3 install beautifulsoup4

You should consider upgrading via the 'c:\users\j_mar\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns
#%matplotlib inline
import re
import time
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests

In [5]:
no_pages = 2

def get_data(pageNo):  
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
    #https://www.amazon.com.br/gp/bestsellers/books?ref_=nav_em__books_best_0_2_22_4
    r = requests.get('https://www.amazon.com.br/gp/bestsellers/books/ref=zg_bs_pg_'+str(pageNo)+'?ie=UTF8&pg='+str(pageNo), headers=headers)#, proxies=proxies)
    content = r.content
    soup = BeautifulSoup(content)
    #print(soup)

    alls = []
    for d in soup.findAll('div', attrs={'class':'a-section a-spacing-none aok-relative'}):
        #print(d)
        name = d.find('span', attrs={'class':'zg-text-center-align'})
        n = name.find_all('img', alt=True)
        #print(n[0]['alt'])
        author = d.find('a', attrs={'class':'a-size-small a-link-child'})

        price = d.find('span', attrs={'class':'p13n-sc-price'})

        imgUrl = d.find('img')['src']
        
        siteUrl = "www.amazon.com.br"
        siteUrl += d.find('a', attrs={'class': 'a-link-normal'})['href']

        all1=[]

        if name is not None:
            #print(n[0]['alt'])
            all1.append(n[0]['alt'])
        else:
            all1.append("unknown-product")

        if author is not None:
            #print(author.text)
            all1.append(author.text)
        elif author is None:
            author = d.find('span', attrs={'class':'a-size-small a-color-base'})
            if author is not None:
                all1.append(author.text)
            else:    
                all1.append('0')
        
        if imgUrl is not None:
            all1.append(imgUrl)
        else:
            all1.append("invalid-image")
        
        if siteUrl is not None:
            all1.append(siteUrl)
        else:
            all1.append("invalid-url-site")

        if price is not None:
            #print(price.text)
            all1.append(price.text)
        else:
            all1.append('0')
        alls.append(all1)    
    return alls

In [6]:
results = []
for i in range(1, no_pages+1):
    results.append(get_data(i))
flatten = lambda l: [item for sublist in l for item in sublist]
df = pd.DataFrame(flatten(results),columns=['Titulo','Autor','ImgUrl','SiteUrl', 'Preco'])
df.to_csv('amazon_bestsellers_products.csv', index=False, encoding='utf-8')

In [7]:
df = pd.read_csv("amazon_bestsellers_products.csv")
df.shape

(50, 5)

In [10]:
df.head(61)

,Titulo,Autor,ImgUrl,SiteUrl,Preco
0,"Pai rico, pai pobre: Edição de 20 anos atualiz...",Robert T. Kiyosaki,https://images-na.ssl-images-amazon.com/images...,www.amazon.com.br/Pai-rico-pai-pobre-anos/dp/8...,"R$ 25,99"
1,O homem mais rico da Babilônia,George S Clason,https://images-na.ssl-images-amazon.com/images...,www.amazon.com.br/Homem-Mais-Rico-Babil%C3%B4n...,"R$ 14,90"
2,Os sete maridos de Evelyn Hugo,Taylor Jenkins Reid,https://images-na.ssl-images-amazon.com/images...,www.amazon.com.br/Os-sete-maridos-Evelyn-Hugo/...,"R$ 29,90"
3,Orgulho e preconceito,Jane Austen,https://images-na.ssl-images-amazon.com/images...,www.amazon.com.br/Orgulho-Preconceito-Jane-Aus...,"R$ 27,39"
4,Especialista em pessoas: Soluções bíblicas e i...,Tiago Brunet,https://images-na.ssl-images-amazon.com/images...,www.amazon.com.br/Especialista-pessoas-Solu%C3...,"R$ 22,38"
5,12 regras para a vida: Um antídoto para o caos,Jordan B. Peterson,https://images-na.ssl-images-amazon.com/images...,www.amazon.com.br/Regras-Para-Vida-Ant%C3%ADdo...,"R$ 24,80"
6,Mulheres que correm com os lobos,Clarissa Pinkola Estés,https://images-na.ssl-images-amazon.com/images...,www.amazon.com.br/Mulheres-que-Correm-com-Lobo...,"R$ 33,99"
7,As obras revolucionárias de George Orwell - Bo...,George Orwell,https://images-na.ssl-images-amazon.com/images...,www.amazon.com.br/As-obras-revolucion%C3%A1ria...,"R$ 24,90"
8,Mais esperto que o Diabo: O mistério revelado ...,Napoleon Hill,https://images-na.ssl-images-amazon.com/images...,www.amazon.com.br/Mais-esperto-que-Diabo-liber...,"R$ 24,90"
9,Torto arado,Itamar Vieira Junior,https://images-na.ssl-images-amazon.com/images...,www.amazon.com.br/Torto-arado-Itamar-Vieira-Ju...,"R$ 35,60"
